## ETL for Diabetes Encounter Data (CSV file)

#### 1. Import Libraries & Load Configuration

This block loads the required libraries and database engine from the config file, ensuring connection parameters are centralized.

In [23]:
import pandas as pd
from config import pg_engine

#### 2. Extract: Read the Diabetes CSV File

The raw clinical dataset is imported into a DataFrame. This acts as the core source for patient encounters, diagnoses, and hospital events.

In [24]:
df = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\Healthcare-data-warehouse\source_data\diabetic_data.csv")

#### 3. Clean & Preprocess Raw Data

This step standardizes missing values, removes unusable rows, enforces data consistency, and recasts numeric fields.
It ensures the dataset is analysis-ready before dimension construction.

In [26]:
df.replace("?", None, inplace=True)
df = df.dropna(subset=["encounter_id", "patient_nbr"])
df = df.drop_duplicates(subset=["encounter_id"])

numeric_cols =     ["time_in_hospital",
    "num_lab_procedures",
    "num_procedures",
    "num_medications",
    "number_outpatient",
    "number_emergency",
    "number_inpatient",
    "number_diagnoses",]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

#### 4. Derive Additional Features

Feature engineering adds business-useful fields such as readmission flags and age groups.
It also aligns column names to match PostgreSQL schemas.

In [27]:
df["readmitted_raw"] = df["readmitted"]
df["readmitted_30d_flag"] = df["readmitted"].apply(lambda x: x == "<30")
df["age_group"] = df["age"]
df["diabetesmed"] = df["diabetesMed"]

Standardizing admission ID datatypes avoids join mismatches later.

In [29]:
for col in ["admission_type_id", "discharge_disposition_id", "admission_source_id"]:
    df[col] = df[col].astype(str)

#### 5. Transform: Build Patient Dimension (dim_patient)

This extracts patient-level attributes and ensures one record per patient.
It forms the foundation for all patient-based analysis.

In [30]:
dim_patient = (
    df[["patient_nbr", "race", "gender", "age_group", "payer_code"]]
    .drop_duplicates(subset=["patient_nbr"])
)
dim_patient["source_system"] = "CSV_diabetes"

#### 6. Transform: Build Admission Dimension (dim_admission)

Admission-related metadata is normalized into a clean lookup table.
This supports analysis of hospital intake patterns.

In [33]:
dim_admission = (
    df[["admission_type_id", "discharge_disposition_id", "admission_source_id"]]
    .drop_duplicates()
)
dim_admission.rename(
    columns={
        "admission_type_id": "admission_type",
        "discharge_disposition_id": "discharge_disposition",
        "admission_source_id": "admission_source",
    },
    inplace=True,
)
dim_admission["source_system"] = "CSV_diabetes"

#### 7. Transform: Build Diagnosis Dimension (dim_diagnosis)

Diagnosis codes from three columns are unpivoted, deduplicated, and categorized by ICD prefix.
This aligns clinical coding into an analytic dimension.

In [34]:
diag_long = (
    pd.melt(
        df[["encounter_id", "diag_1", "diag_2", "diag_3"]],
        id_vars=["encounter_id"],
        value_vars=["diag_1", "diag_2", "diag_3"],
        var_name="diag_position",
        value_name="diagnosis_code",
    )
    .dropna(subset=["diagnosis_code"])
    .drop_duplicates(subset=["diagnosis_code"])
)
diag_long["icd_category"] = diag_long["diagnosis_code"].str.slice(0, 3)
dim_diagnosis = diag_long[["diagnosis_code", "icd_category"]].drop_duplicates()
dim_diagnosis["source_system"] = "CSV_diabetes"

#### 8. Load Dimensions into PostgreSQL

To maintain repeatable ETL runs, dimension tables are truncated and reloaded.
This ensures consistency and avoids leftover data.

In [36]:
with pg_engine.begin() as conn:
    # Make the load repeatable: clear dims first
    conn.exec_driver_sql("TRUNCATE TABLE dim_diagnosis RESTART IDENTITY CASCADE;")
    conn.exec_driver_sql("TRUNCATE TABLE dim_admission RESTART IDENTITY CASCADE;")
    conn.exec_driver_sql("TRUNCATE TABLE dim_patient RESTART IDENTITY CASCADE;")

    dim_patient.to_sql("dim_patient", con=conn, if_exists="append", index=False)
    dim_admission.to_sql("dim_admission", con=conn, if_exists="append", index=False)
    dim_diagnosis.to_sql("dim_diagnosis", con=conn, if_exists="append", index=False)

#### 9. Extract Dimension Keys for Fact Table Construction

The ETL retrieves dimension tables to build mapping dictionaries.
These allow efficient FK assignment in the fact table.

In [37]:
# Re-read dimensions with keys to build fact
with pg_engine.connect() as conn:
    dim_patient_db = pd.read_sql("SELECT * FROM dim_patient", conn)
    dim_admission_db = pd.read_sql("SELECT * FROM dim_admission", conn)
    dim_diag_db = pd.read_sql("SELECT * FROM dim_diagnosis", conn)

patient_key_map = dim_patient_db.set_index("patient_nbr")["patient_key"].to_dict()
admission_key_map = (
    dim_admission_db
    .set_index(["admission_type", "discharge_disposition", "admission_source"])["admission_dim_key"]
    .to_dict()
)
diag_key_map = dim_diag_db.set_index("diagnosis_code")["diagnosis_key"].to_dict()

#### 10. Assign Foreign Keys to the Fact Data

Keys from the dimension tables are mapped into the main encounter dataset.
This links clinical events to the dimensional model.

In [38]:
df["patient_key"] = df["patient_nbr"].map(patient_key_map)
df["admission_dim_key"] = df.apply(
    lambda row: admission_key_map.get(
        (row["admission_type_id"],
         row["discharge_disposition_id"],
         row["admission_source_id"])
    ),
    axis=1,
)
df["primary_diagnosis_key"] = df["diag_1"].map(diag_key_map)
df["secondary_diagnosis_key"] = df["diag_2"].map(diag_key_map)
df["tertiary_diagnosis_key"] = df["diag_3"].map(diag_key_map)

#### 11. Select Required Fact Table Columns

A clean fact record is created containing keys, clinical measures, and outcome indicators.

In [39]:
fact_cols = [
    "encounter_id",
    "patient_key",
    "admission_dim_key",
    "primary_diagnosis_key",
    "secondary_diagnosis_key",
    "tertiary_diagnosis_key",
    "time_in_hospital",
    "num_lab_procedures",
    "num_procedures",
    "num_medications",
    "number_outpatient",
    "number_emergency",
    "number_inpatient",
    "number_diagnoses",
    "readmitted_raw",
    "readmitted_30d_flag",
    "change",
    "diabetesmed",
]

fact_df = df[fact_cols].copy()

#### 12. Filter Invalid Records

Encounter rows missing patient foreign keys are removed to enforce relational integrity.

In [40]:
# Keep only rows with a valid patient_key
fact_df = fact_df.dropna(subset=["patient_key"])

#### 13. Load Fact Table into PostgreSQL

The fact table is truncated for clean reloading and populated with the transformed records.

In [41]:
fact_df["source_system"] = "CSV_diabetes"

with pg_engine.begin() as conn:
    conn.exec_driver_sql("TRUNCATE TABLE fact_hospital_admission_parted RESTART IDENTITY;")

    fact_df.to_sql(
        "fact_hospital_admission_parted",
        con=conn,
        if_exists="append",
        index=False,
    )